In [11]:
!pip -q install flask-ngrok
!mkdir -p /root/.ngrok2/
!echo "authtoken: 2bfWmDGEi2kchNrENYrDuPiEmaZ_3Z7s5gsteZ1UeksBZTVnU" > /root/.ngrok2/ngrok.yml

In [12]:
# prompt: google drive mount

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from flask import jsonify, request
from flask import Flask
from flask_ngrok import run_with_ngrok
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import json
from io import BytesIO
import base64
import requests

# 모델 정의
class CustomResNet18(nn.Module):
    def __init__(self, num_classes=3):
        super(CustomResNet18, self).__init__()
        self.resnet18 = models.resnet18(pretrained=False)
        in_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet18(x)

# 모델 인스턴스 생성
model = CustomResNet18(num_classes=3)

# 학습된 가중치 로드
state_dict = torch.load("/content/drive/MyDrive/AIR/resnet_pre_epoch_7.pth")
# 모델의 state_dict와 불러온 state_dict의 키를 비교해 일치하도록 수정
model_dict = model.state_dict()
state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
model_dict.update(state_dict)
model.load_state_dict(model_dict)
model.eval()


# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])




app = Flask(__name__)
run_with_ngrok(app)

@app.route('/img', methods=["POST", "GET"])
def home():
    dict_data = request.get_json()

    img = dict_data['img']
    img = base64.b64decode(img)
    img = BytesIO(img)
    img = Image.open(img)

    # img.save('test.jpg')
    input_tensor = transform(img)
    input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

    # Model prediction
    with torch.no_grad():
        output = model(input_batch)

    # Get predicted class
    _, predicted_class = torch.max(output, 1)

    if predicted_class.item()==0:
        return_val="tower"
    elif predicted_class.item()==1:
        return_val="wing"
    else:
        return_val="dual"



    # Send a POST request to another server with the predicted class
    url = 'http://33a9-34-141-212-81.ngrok-free.app/chat'
    data = {"input": return_val}
    headers = {'Content-Type': 'application/json'}

    # Send a POST request to the second server
    r=requests.post(url, json=data, headers=headers)

    response_data = r.json()

    # Get the value associated with the 'output' key
    output_value = response_data.get('output', '')


    return jsonify({"output": output})

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://2847-35-245-211-53.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [02/Feb/2024 05:11:12] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Feb/2024 05:11:13] "GET /favicon.ico HTTP/1.1" 404 -
